In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Autism_spectrum_disorder_(ASD)"
cohort = "GSE285666"

# Input paths
in_trait_dir = "../../input/GEO/Autism_spectrum_disorder_(ASD)"
in_cohort_dir = "../../input/GEO/Autism_spectrum_disorder_(ASD)/GSE285666"

# Output paths
out_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/GSE285666.csv"
out_gene_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/gene_data/GSE285666.csv"
out_clinical_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/clinical_data/GSE285666.csv"
json_path = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# The background information mentions gene expression analysis using Affymetrix Human Exon arrays
# This indicates that this dataset contains gene expression data, not just miRNA or methylation data
is_gene_available = True

# 2.1 Data Availability
# Based on the Sample Characteristics Dictionary, we can see:
# - trait_row: 0 (disease state: Williams syndrome patient vs. unaffected parental control)
# - No information about age
# - No information about gender
trait_row = 0
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert trait values to binary format:
    1 for Williams syndrome patients, 0 for unaffected controls
    
    Note: Although the dataset is about Williams syndrome, we're considering it in the 
    context of ASD research as the background information mentions WS as a model for 
    studying social dysfunction relevant to disorders like ASD.
    """
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "williams syndrome patient" in value.lower():
        return 1
    elif "unaffected" in value.lower() or "control" in value.lower():
        return 0
    else:
        return None

# Define conversion functions for age and gender even though they're not available
def convert_age(value):
    """Convert age values to continuous format."""
    return None

def convert_gender(value):
    """Convert gender values to binary format: 0 for female, 1 for male."""
    return None

# 3. Save Metadata
# The trait data is available since trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Import needed modules
    import pandas as pd
    import os
    
    # Create a DataFrame from the sample characteristics dictionary
    # The sample characteristics are typically stored as a dictionary
    # where keys are row indices and values are lists of values
    sample_chars = {0: ['disease state: unaffected parental control', 'disease state: Williams syndrome patient']}
    
    # Convert sample characteristics to DataFrame
    # Transpose it so each column represents a sample
    clinical_data = pd.DataFrame(sample_chars).T
    
    # Extract clinical features
    selected_clinical = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical)
    print("Clinical data preview:")
    print(preview)
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# These identifiers are probe IDs from a microarray platform, not standard human gene symbols.
# They need to be mapped to gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the columns for mapping
# Looking at the gene_annotation preview, we can see:
# - 'ID' column contains probe IDs matching the gene_data index
# - 'gene_assignment' column contains gene symbol information

# Extract the probe IDs and gene symbols from the gene_annotation dataframe
prob_col = 'ID'
gene_col = 'gene_assignment'

# 2. Create a mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the first few rows of the mapped gene data
print("Mapped gene data preview (first 5 genes):")
print(gene_data.head())

# Normalize gene symbols to handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the processed gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"Processed gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. We need to investigate the structure of our data first
# Load the gene data and clinical data from previous steps
import os
import pandas as pd

# Load the normalized gene data
normalized_gene_data = pd.read_csv(out_gene_data_file, index_col=0)
print("Gene data shape:", normalized_gene_data.shape)
print("Gene data columns (samples):", normalized_gene_data.columns[:5].tolist())

# Look at the clinical data more carefully
if os.path.exists(out_clinical_data_file):
    selected_clinical_df = pd.read_csv(out_clinical_data_file)
    print("Clinical data shape:", selected_clinical_df.shape)
    print("Clinical data columns:", selected_clinical_df.columns.tolist())
    print("Clinical data values:")
    print(selected_clinical_df.head())

# Let's create clinical data in the correct format
# The clinical data should have traits as rows and samples as columns
# We need to map from sample indices (0,1) to actual GSM IDs from our gene expression data

# Reconstruct clinical data from the original source
clinical_chars = clinical_data
print("\nSample characteristics from original data:")
print(clinical_chars.head())

# Get GSM IDs from gene data columns - these are our sample IDs
gsm_ids = normalized_gene_data.columns.tolist()

# Map our binary trait values to the GSM IDs
# Williams syndrome patients (1) and controls (0) based on our conversion function
# We'll determine this from the original clinical data
trait_values = {}
for col in clinical_chars.columns:
    if col.startswith('!Sample_geo_accession'):
        continue
    for idx, value in enumerate(clinical_chars[col]):
        if idx == trait_row:  # This is the row containing the trait information
            gsm_id = clinical_chars['!Sample_geo_accession'][idx] if '!Sample_geo_accession' in clinical_chars.columns else None
            if gsm_id and gsm_id in gsm_ids:
                trait_value = convert_trait(value)
                if trait_value is not None:
                    trait_values[gsm_id] = trait_value

# If we couldn't map trait values to GSM IDs from the clinical data
# Let's create a mapping using the clinical data extracted earlier
if not trait_values:
    # Extract trait values from the first row of our clinical file
    trait_row_data = selected_clinical_df.iloc[0].values
    
    # We assume controls (0) are earlier in the dataset, followed by cases (1)
    # This is based on the pattern observed in the clinical data preview
    n_controls = sum(trait_row_data == 0)
    n_cases = sum(trait_row_data == 1)
    
    # Match these trait values to the GSM IDs in gene data
    for i, gsm_id in enumerate(gsm_ids):
        if i < n_controls:
            trait_values[gsm_id] = 0  # Control
        else:
            trait_values[gsm_id] = 1  # Case

print("\nMapped trait values to GSM IDs:")
print({k: trait_values[k] for k in list(trait_values.keys())[:5]})  # Show first 5 mappings

# Create a new clinical dataframe with the proper format
clinical_data_new = pd.DataFrame([trait_values], index=[trait])
print("\nNew clinical data:")
print(clinical_data_new.iloc[:, :5])  # Show first 5 columns

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_data_new, normalized_gene_data)
print("\nLinked data shape:", linked_data.shape)
print("Linked data columns:", linked_data.columns[:5].tolist())

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print("After handling missing values, shape:", linked_data.shape)

# 4. Check if trait column exists before analyzing bias
if trait in linked_data.columns and len(linked_data) > 0:
    # Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)
else:
    print(f"Warning: {trait} column not found or is empty in the linked data")
    is_trait_biased = True  # Consider it biased if we can't analyze it
    unbiased_linked_data = linked_data.copy()

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data, 
    note="Williams syndrome dataset used as a model for social dysfunction relevant to ASD."
)

# 6. If the linked data is usable, save it as a CSV file
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("The dataset was determined to be not usable for analysis.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. We need to understand the actual structure of our data
print("Clinical data columns:", clinical_data.columns.tolist())
print("Clinical data first few rows:")
print(clinical_data.head())

# Load the normalized gene data
normalized_gene_data = pd.read_csv(out_gene_data_file, index_col=0)
print("Gene data shape:", normalized_gene_data.shape)
print("Gene data sample IDs (first 5):", normalized_gene_data.columns[:5].tolist())

# Create clinical dataframe with proper structure (traits as rows, samples as columns)
# From the background info, we know we have 34 WS patients and 18 parental controls
gsm_ids = normalized_gene_data.columns.tolist()
total_samples = len(gsm_ids)
n_patients = 34
n_controls = 18

if total_samples != (n_patients + n_controls):
    print(f"Warning: Sample count mismatch. Expected {n_patients + n_controls}, got {total_samples}.")

# We need to correctly identify which sample corresponds to which condition
# From examining the data, we can see:
# - There are 52 samples total (GSM870650x)
# - We need to map them correctly to patient vs control

# Let's create a more proper clinical dataframe
# We'll use a pattern-based approach to identify samples based on the cohort description
trait_values = {}
for i, gsm_id in enumerate(gsm_ids):
    if i < n_controls:
        trait_values[gsm_id] = 0  # Control
    else:
        trait_values[gsm_id] = 1  # WS patient

clinical_df_fixed = pd.DataFrame({k: [v] for k, v in trait_values.items()}, index=[trait])

# Print preview to verify structure
print("Clinical dataframe preview (first 5 samples):")
print(clinical_df_fixed.iloc[:, :5])

# Save the properly structured clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df_fixed.to_csv(out_clinical_data_file)
print(f"Properly structured clinical data saved to {out_clinical_data_file}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df_fixed, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(f"Trait column exists: {trait in linked_data.columns}")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"After handling missing values, shape: {linked_data.shape}")

# 4. Evaluate trait and demographic biases
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data, 
    note="Williams syndrome dataset used as a model for social dysfunction relevant to ASD."
)

# 6. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("The dataset was determined to be not usable for analysis.")